Emotion classification
================

In [6]:
"""
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable

from src.classifiers.classification_cnn import ClassificationCNN
from src.data_utils import get_CIFAR10_datasets, OverfitSampler, rel_error

#torch.set_default_tensor_type('torch.FloatTensor')

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
"""
pass

Dataset
=========
Preprocessing will be needed

In [7]:
# Load the (preprocessed) CIFAR10 data. The preprocessing includes
# channel swapping, normalization and train-val-test splitting.
# Loading the datasets might take a while.

"""
train_data, val_data, test_data, mean_image = get_CIFAR10_datasets()
print("Train size: %i" % len(train_data))
print("Val size: %i" % len(val_data))
print("Test size: %i" % len(test_data))
"""
pass

Examples
------------------

In [8]:
"""
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for cls_idx, cls in enumerate(classes):
    cls_data = [datum for datum in test_data if datum[1] == cls_idx]
    rnd_idxs = np.random.randint(0, len(cls_data), samples_per_class)
    rnd_cls_data = [datum for i, datum in enumerate(cls_data) if i in rnd_idxs]
    for i, cls_datum in enumerate(rnd_cls_data):
        plt_idx = i * num_classes + cls_idx + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(cls_datum[0].numpy().transpose(1,2,0) + mean_image.transpose(1,2,0))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()
"""
pass

## Training the Network

In [ ]:
from train_classifier import train

train()


Loading data...
START TRAIN.
[Iteration 10/2200] TRAIN loss: 2.136
[Iteration 20/2200] TRAIN loss: 2.054
[Iteration 30/2200] TRAIN loss: 2.082
[Iteration 40/2200] TRAIN loss: 2.055
[Epoch 1/50] TRAIN acc/loss: 0.080/1.860
[Epoch 1/50] VAL   acc/loss: 0.038/2.075
[Iteration 54/2200] TRAIN loss: 2.010
[Iteration 64/2200] TRAIN loss: 2.073
[Iteration 74/2200] TRAIN loss: 2.042
[Iteration 84/2200] TRAIN loss: 2.059
[Epoch 2/50] TRAIN acc/loss: 0.160/1.922
[Epoch 2/50] VAL   acc/loss: 0.062/2.016


## Visualizing Results
Loss and accuracy

In [ ]:
import matplotlib.pyplot as plt

plt.subplot(2, 1, 1)
plt.plot(solver.train_loss_history, 'o')
plt.plot(range(0, len(solver.val_loss_history) * 19, 19), solver.val_loss_history, '-o')
plt.xlabel('iteration')
plt.ylabel('loss')

plt.subplot(2, 1, 2)
plt.plot(solver.train_acc_history, '-o')
plt.plot(range(0, len(solver.val_acc_history) * 19, 19), solver.val_acc_history, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

Filter visualization

In [ ]:
from src.vis_utils import visualize_grid

# first (next) parameter should be convolutional
conv_params = next(model.parameters()).data.cpu().numpy()
grid = visualize_grid(conv_params.transpose(0, 2, 3, 1))
plt.imshow(grid.astype('uint8'))
plt.axis('off')
plt.gcf().set_size_inches(6, 6)
plt.show()

# Testing the Model

In [ ]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=50, shuffle=False, num_workers=4)

scores = []
for inputs, target in test_loader:
    inputs, targets = Variable(inputs), Variable(target)
    if model.is_cuda:
        inputs, targets = inputs.cuda(), targets.cuda()

    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    preds = preds.type(torch.IntTensor).cpu()
    targets = targets.cpu()
    scores.extend((preds == targets).data.numpy())
    
print('Test set accuracy: %f' % np.mean(scores))

## Save the Model

When you are satisfied with your training, you can save the model.

In [ ]:
model.save("models/classification_cnn.model")